In [ ]:
# !pip install pymysql easydict boto3 python-dotenv

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import os

os.environ

environ{'PATH': '/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'HOSTNAME': '3043c1ddafa1',
        'TERM': 'xterm-color',
        'LANG': 'C.UTF-8',
        'GPG_KEY': 'E3FF2839C048B25C084DEBE9B26995E310250568',
        'PYTHON_VERSION': '3.9.16',
        'PYTHON_PIP_VERSION': '22.0.4',
        'PYTHON_SETUPTOOLS_VERSION': '58.1.0',
        'PYTHON_GET_PIP_URL': 'https://github.com/pypa/get-pip/raw/1a96dc5acd0303c4700e02655aefd3bc68c78958/public/get-pip.py',
        'PYTHON_GET_PIP_SHA256': 'd1d09b0f9e745610657a528689ba3ea44a73bd19c60f4c954271b790c71c2653',
        'HOME': '/root',
        'PYDEVD_USE_FRAME_EVAL': 'NO',
        'JPY_PARENT_PID': '1',
        'CLICOLOR': '1',
        'FORCE_COLOR': '1',
        'CLICOLOR_FORCE': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://matplotlib_inline.backend_inline'}

In [2]:
from easydict import EasyDict

settings = EasyDict()

settings.AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
settings.AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
settings.REGION_NAME = os.environ["REGION_NAME"]
settings.AWS_ACCOUNT_ID = os.environ["AWS_ACCOUNT_ID"]

settings.DB_HOST = os.environ["DB_HOST"]
settings.DB_USER = os.environ["DB_USER"]
settings.DB_PASSWORD = os.environ["DB_PASSWORD"]
settings.DB_NAME = os.environ["DB_NAME"]
settings.DB_PORT = os.environ["DB_PORT"]

settings

KeyError: 'AWS_ACCESS_KEY_ID'

In [8]:
import pymysql
import pandas as pd 

db = pymysql.connect(
    host=settings.DB_HOST,
    user=settings.DB_USER,
    passwd=settings.DB_PASSWORD,
    db=settings.DB_NAME,
    port=int(settings.DB_PORT)
)

cursor = db.cursor()
cursor

In [9]:
def get_query_to_df(sql: str):
    cursor.execute(sql)
    data = cursor.fetchall()
    columns = [elem[0] for elem in cursor.description]
    return pd.DataFrame(data, columns=columns)

df = get_query_to_df("SELECT * FROM hotel LIMIT 10;")
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,None,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,None,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,None,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,None,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,None,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [21]:
period_df = get_query_to_df("SELECT DISTINCT arrival_date_year, arrival_date_month FROM hotel")
period_df.head()

,arrival_date_year,arrival_date_month
0,2015,July
1,2015,August
2,2015,September
3,2015,October
4,2015,November


In [30]:
# queries = list()
# for _, row in period_df.iterrows():
#     year, month = row["arrival_date_year"], row["arrival_date_month"]
#     sql = f"SELECT * FROM hotel WHERE arrival_date_year={year} && arrival_date_month='{month}'"
#     queries.append(sql)
queries = [
    f"SELECT * FROM hotel WHERE arrival_date_year={row.arrival_date_year} && arrival_date_month='{row.arrival_date_month}'" for _, row in period_df.iterrows()
]

queries

["SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='July'",
 "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='August'",
 "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='September'",
 "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='October'",
 "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='November'",
 "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='December'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='January'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='February'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='March'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='April'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='May'",
 "SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month

In [39]:
import boto3

s3_client = boto3.client(
    service_name="s3",
    aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
    region_name=settings.REGION_NAME,
)

s3_client

In [40]:
from smart_open import open as s_open

# smart_open library로 업로드: 메모리 상의 데이터를 로컬 파일 저장 없이 바로 업로드
# file_name = f"s3://{settings.AWS_BUCKET_NAME}/glue-test/teacher.parquet"
def upload_s3_df(df: pd.DataFrame, file_name: str):
    try:
        with s_open(file_name, "wb", transport_params=dict(client=s3_client)) as out_file:
            df.to_parquet(out_file, engine="pyarrow", compression="gzip", index=False)
        return True
    except Exception as e:
        print("Error occured: ", str(e))
        return False

In [41]:
from datetime import datetime

queries = list()
for _, row in period_df.iterrows():
    year, month = row["arrival_date_year"], row["arrival_date_month"]
    sql = f"SELECT * FROM hotel WHERE arrival_date_year={year} && arrival_date_month='{month}'"
    queries.append(((year, month), sql))

root_path = "teacher"
# df 저장 -> s3://genia-bucket/raw/student0/hotel/yyyy=%d%d%d%d/mm=%d%d/hotel.paquet
for (yyyy, mm), sql in queries:
    tmp_df = get_query_to_df(sql)
    mm = datetime.strptime(mm, "%B").strftime("%m")
    s3_path = f"s3://genia-bucket/raw/{root_path}/hotel/yyyy={yyyy}/mm={mm}/hotel.paquet"
    result = upload_s3_df(tmp_df, s3_path)
    print("result: ", result, sql)

result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='July'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='August'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='September'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='October'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='November'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='December'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='January'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='February'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='March'
result:  True SELECT * FROM hotel WHERE arrival_date_year=2016 && arrival_date_month='April'
result:  True SELECT * FROM hotel WHERE arrival_date_

In [42]:
athena_client = boto3.client(
    service_name="athena",
    aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
    region_name=settings.REGION_NAME,
)

athena_client

## 왜 안될까요??

In [58]:
output_location = "athena/quries"
athena_database = "mini_db"
settings.AWS_BUCKET_NAME = "genia-bucket"

def get_athena_query_exec_id(sql: str):
    response = athena_client.start_query_execution(
        QueryString=sql,
        QueryExecutionContext={"Database": athena_database},
        ResultConfiguration={"OutputLocation": f"s3://genia-bucket/{output_location}/"},
    )
    return response["QueryExecutionId"]

def get_query_result(query_result_path: str):
    obj = s3_client.get_object(
        Bucket=settings.AWS_BUCKET_NAME,
        Key=query_result_path
    )
    return pd.read_csv(obj["Body"])

sample_sql = 'SELECT * FROM "mini_db"."teacher-hotel" limit 1000;'

# Athena Query 실행 요청
query_exec_id = get_athena_query_exec_id(sample_sql)

#  Athena Query 결과 불러오기
query_result_path = f"{output_location}/{query_exec_id}.csv"
sample_df = get_query_result(query_result_path)

sample_df.head()

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

## 왜 될까요??

In [59]:
sample_df = get_query_result(query_result_path)

sample_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,1,1,2016,November,45,1,0,1,1,...,NaN,0,Transient,75.0,0,0,No-Show,2016-11-01,2016,11
1,Resort Hotel,1,378,2016,November,45,1,0,2,2,...,NaN,0,Transient,46.0,0,0,Canceled,2016-10-18,2016,11
2,Resort Hotel,1,378,2016,November,45,1,0,2,2,...,NaN,0,Transient,46.0,0,0,Canceled,2016-03-11,2016,11
3,Resort Hotel,1,378,2016,November,45,1,0,2,2,...,NaN,0,Transient,46.0,0,0,Canceled,2016-10-18,2016,11
4,Resort Hotel,1,378,2016,November,45,1,0,2,2,...,NaN,0,Transient,46.0,0,0,Canceled,2016-10-18,2016,11
